<a href="https://colab.research.google.com/github/Neelll1705/Eye-Disease-Classification---ML/blob/main/EyeDiseaseDetectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification

In [ ]:
!unzip /content/eye-diseases-classification.zip

## Import the library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import PIL
import os

## Load the dataset

In [ ]:
glaucoma = Path('/content/dataset/glaucoma')
cataract = Path('/content/dataset/cataract')
normal = Path('/content/dataset/normal')
diabetic_retinopathy = Path('/content/dataset/diabetic_retinopathy')

In [ ]:
from tqdm import tqdm

disease_type = [glaucoma, cataract,normal,diabetic_retinopathy]
df = pd.DataFrame()

for types in disease_type:
    for imagepath in tqdm(list(types.iterdir()), desc= str(types)):
        df = pd.concat([df, pd.DataFrame({'image': [str(imagepath)],'disease_type': [disease_type.index(types)]})], ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.disease_type.value_counts()

In [ ]:
df.shape

In [ ]:
df['disease_type'] = df['disease_type'].map({0:'glaucoma',1:'cataract',2:'normal',3:'diabetic_retinopathy'})

In [ ]:
df.head()

## Visualizing Images in the dataset

In [ ]:
from PIL import Image
import random

rand_index = random.randint(0, len(df['image']) - 1)
disease_type = df['disease_type'][rand_index]
IMG = Image.open(df['image'][rand_index])
plt.title(f'Disease: {disease_type}')
plt.axis('off')
plt.imshow(IMG)
plt.show()

In [ ]:
df1 = df.sample(frac=1).reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
for i in range (50):
    plt.subplot(5,10,i+1)
    img = Image.open(df1['image'][i])
    plt.title(f' Disease {df1.disease_type[i]}')
    plt.imshow(img);
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
sns.countplot(x = 'disease_type', data = df1)
plt.xlabel("Disease type")
plt.show()

## Model training

In [ ]:
#import necessary libraries for the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data_dir='dataset'

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.25,
    subset="training",
    seed=123,
    image_size=(128,128),
    batch_size=32,
    label_mode='categorical'  # Use categorical labels
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.25,
    subset="validation",
    seed=123,
    image_size=(128,128),
    batch_size=32,
    label_mode='categorical'
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
num_classes = 4

In [ ]:
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(128,128, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=20)

## Model Performance

In [ ]:
# create a plot of accuracy and val_acuracy for each epoch
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'])
plt.show()

In [ ]:
# create a plot of loss and val_loss for each epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'])
plt.show()

In [ ]:
# Get the class names
class_names = train_ds.class_names

# Initialize variables to store class-wise counts
class_counts = {class_name: 0 for class_name in class_names}
class_correct_counts = {class_name: 0 for class_name in class_names}

# Iterate over the validation dataset to calculate class-level accuracy
for images, labels in val_ds:
    predictions = model.predict(images)
    predicted_classes = tf.argmax(predictions, axis=1)
    true_classes = tf.argmax(labels, axis=1)

    for i in range(len(class_names)):
        class_name = class_names[i]
        class_counts[class_name] += tf.reduce_sum(tf.cast(true_classes == i, tf.int32))
        class_correct_counts[class_name] += tf.reduce_sum(tf.cast(tf.logical_and(true_classes == i, true_classes == predicted_classes), tf.int32))

# Calculate and print class-level accuracy
print('Class-level accuracy:')
for class_name in class_names:
    accuracy = class_correct_counts[class_name] / class_counts[class_name]
    print(f'{class_name}: {accuracy.numpy():.4f}')

## Save the Model

In [ ]:
#finally save the model

tf.keras.models.save_model(model,'EyeModel.h5')

## Test the model

In [ ]:
from tensorflow.keras.preprocessing import image
img = image.load_img('/content/dataset/cataract/1062_right.jpg',target_size =(128,128))
img

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred =np.argmax(model.predict(x))
op = {0:'cataract',1:'diabetic_retinopathy',2:'glaucoma',3:'normal'}
op[pred]

In [ ]:
img = image.load_img('/content/dataset/diabetic_retinopathy/10009_right.jpeg',target_size =(128,128))
img

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred =np.argmax(model.predict(x))
op = {0:'cataract',1:'diabetic_retinopathy',2:'glaucoma',3:'normal'}
op[pred]